In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

In [4]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-04 07:18:37,020 INFO: Initializing external client
2025-03-04 07:18:37,021 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 07:18:38,825 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215665
Fetching data from 2025-02-03 12:18:37.016714+00:00 to 2025-03-04 11:18:37.016714+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.32s) 


In [5]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-03 13:00:00+00:00,2,0
1,2025-02-03 14:00:00+00:00,2,0
2,2025-02-03 15:00:00+00:00,2,0
3,2025-02-03 16:00:00+00:00,2,0
4,2025-02-03 17:00:00+00:00,2,0
...,...,...,...
174440,2025-03-04 07:00:00+00:00,263,130
174441,2025-03-04 08:00:00+00:00,263,152
174442,2025-03-04 09:00:00+00:00,263,134
174443,2025-03-04 10:00:00+00:00,263,116


In [6]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174445 entries, 0 to 178460
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         174445 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  174445 non-null  int32                  
 2   rides               174445 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.0 MB


In [7]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [8]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174445 entries, 0 to 178460
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         174445 non-null  datetime64[us]
 1   pickup_location_id  174445 non-null  int32         
 2   rides               174445 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.0 MB


In [9]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [10]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,32,48,92,76,8,155,62,60,80,6,...,61,14,13,5,21,120,218,146,263,2025-02-14 16:00:00
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,210,2025-02-06 12:00:00
2,3,0,0,0,0,2,0,2,0,1,...,1,1,3,0,2,1,2,1,216,2025-02-04 22:00:00
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,147,2025-02-05 02:00:00
4,3,2,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,217,2025-02-26 14:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,4,0,1,0,...,0,0,1,0,0,0,1,1,86,2025-02-25 16:00:00
247,0,0,1,0,0,0,2,2,0,1,...,1,0,1,3,0,0,0,0,62,2025-02-24 09:00:00
248,243,130,148,572,217,231,173,141,160,338,...,140,10,0,133,311,225,168,179,142,2025-02-17 00:00:00
249,6,5,22,5,23,2,6,1,14,28,...,15,1,0,4,0,24,9,4,74,2025-03-01 05:00:00


In [11]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-04 07:18:54,626 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 07:18:54,647 INFO: Initializing external client
2025-03-04 07:18:54,647 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 07:18:56,020 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215665
Fetching data from 2025-02-03 12:18:54.626548+00:00 to 2025-03-04 11:18:54.626548+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.90s) 


In [12]:
current_date

Timestamp('2025-03-04 12:18:54.626548+0000', tz='Etc/UTC')

In [13]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-03 13:00:00
1,0,0,0,0,0,1,0,0,0,0,...,0,1,1,1,0,0,0,0,3,2025-03-03 13:00:00
2,2,4,4,1,4,0,1,3,3,6,...,1,1,7,11,4,3,5,1,4,2025-03-03 13:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,6,2025-03-03 13:00:00
4,2,2,4,1,1,1,5,1,2,0,...,1,2,2,5,3,3,2,8,7,2025-03-03 13:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,1,0,0,0,0,0,0,0,0,...,0,2,0,2,0,0,0,1,259,2025-03-03 13:00:00
247,2,4,3,3,1,1,3,0,1,0,...,1,5,0,3,0,0,1,4,260,2025-03-03 13:00:00
248,25,37,25,38,30,41,24,17,16,11,...,2,2,2,9,12,7,16,25,261,2025-03-03 13:00:00
249,70,71,102,81,98,73,43,34,22,4,...,16,69,139,172,124,118,98,79,262,2025-03-03 13:00:00


In [14]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-04 07:19:12,319 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 07:19:12,331 INFO: Initializing external client
2025-03-04 07:19:12,334 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 07:19:13,532 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215665


In [15]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=0.6 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


In [16]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,3.0
3,6,0.0
4,7,3.0
...,...,...
246,259,0.0
247,260,2.0
248,261,29.0
249,262,78.0


In [17]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([237, 161, 236, 132, 138, 162, 142, 186, 239, 140])